In [1]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import time

In [2]:
# MySQL Credentials
host = 'localhost'
user = 'root'
password = 'V@nya'
port = 3306
database_name = 'Play_Market_2025'

In [3]:
table_1 = 'apps_info'
table_2 = 'apps_reviews'
table_3 = 'games_info'
table_4 = 'games_reviews'

# App performance & Downloads
# Top 10 apps by total downloads
query_1A = f"""
            select app_id, app_name, categories, downloads from {table_1}
            group by 1,2
            order by 4 desc
            limit 10;
            """

# Average Score & number of reviews per category
query_2A = f"""
            select t1.categories, count(t2.review_text) as "Total Reviews", avg(t2.review_score) as "Average Review Score" from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            group by 1
            order by 2 desc;
            """

# Monthly trend of app reviews
query_3A = f"""
            select date_format(review_date, '%y-%m') as "Month_Name", avg(review_score) as "Average_Review_Score" from {table_2}
            where monthname(review_date) is not null
            group by 1
            order by 1;
            """

# Top 5 most helpful app reviews
query_4A = f"""
            select t1.app_id, t1.app_name, count(t2.review_score) as "Number of Review Scores", sum(t2.helpful_count) as "Total helpful app reviews" from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            group by 1,2
            order by 4 desc
            limit 5;
            """

In [4]:
#Game Insights
#Top 10 games with highest rating and more than 10,000 downloads.
query_1G = f"""
            select game_id, game_name, downloads, max(score) as "Highest_rating" from {table_3}
            where downloads > 10000
            group by 1,2,3
            order by 4 desc
            limit 10;
            """
#Game section with average score and number of games
query_2G = f"""
            select section, count(game_name) as "Number of games", avg(score) as "Average_Score" from {table_3}
            group by 1
            order by 2 desc;
            """

#Monthly trend of game reviews
query_3G = f"""
            select date_format(review_date, '%y-%m') as "Month_Name", avg(review_score) as "Average_Review_Score" from {table_4}
            where monthname(review_date) is not null
            group by 1
            order by 1;
            """

In [5]:
#User Review Analysis
#Apps with the highest average review score
query_1U = f"""
            select t1.app_name, avg(t2.review_score) as "Average_Review_score" from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            group by 1
            order by 2 desc
            limit 10;
            """

#Total reviews and average helpful count per app/game
query_2U = f"""
            select "Apps" as "Type", t1.app_name, count(t2.review_score) as "Total Reviews", avg(t2.helpful_count) as "Average Helpful count" from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            group by 2
            Union ALL
            select "Games" as "Type", t1.game_name, count(t2.review_score) as "Total Reviews", avg(t2.helpful_count) as "Average Helpful count" from {table_3} t1
            join {table_4} t2 on t1.game_id = t2.game_id
            group by 2;
            """

#Top apps and games by total review volume
query_3UA = f"""
            select "App" as "Type", t1.app_name, count(t2.review_score) as "Total_Review_Volume" from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            group by 1,2
            order by 3 desc
            limit 10;
            """

query_3UB = f"""
            select "Games" as "Type", t1.game_name, count(t2.review_score) as "Total_Review_Volume" from {table_3} t1
            join {table_4} t2 on t1.game_id = t2.game_id
            group by 1,2
            order by 3 desc
            limit 10;
            """

In [6]:
#Dynamic_charting

#Filter by category: Show App Performance
query_1DC = f"""
            select categories, app_name, sum(downloads) as "Total_downloads", Avg(score) as "Average_Score" from {table_1}
            group by 1,2;
            """

#Compare App Review Scores by Section
query_2DC = f"""
            select t1.section, t1.app_name, AVG(t2.review_score) as "avg_score" from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            group by 1,2;
            """

#Review Trend Over Time by App
query_3DC = f"""
            select t2.app_id, t2.app_name, DATE_FORMAT(review_date, '%y-%m'), count(*) as "Total_Review" from {table_2} t1
            join {table_1} t2 on t1.app_id = t2.app_id
            group by 1,2,3
            order by 4;
            """

#Select a Game Section and Compare Downloads
query_4DC = f"""
            select section, game_name, downloads from {table_3};
            """

#Helpful Review Analysis per App/Game
query_5DC_A = f"""
            select t1.app_name, avg(t2.helpful_count) as "Average_helpful_count" from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            group by 1
            order by 2 desc;
            """

query_5DC_B = f"""
              select t1.game_name, avg(t2.helpful_count) as "Average_helpful_count" from {table_3} t1
              join {table_4} t2 on t1.app_id = t2.app_id
              group by 1
              order by 2 desc;
              """

#Download Vs Score Scatter Plot for selected category
query_6DC = f"""
            select categories, app_name, downloads, score from {table_1};
            """

#Top 5 reviewed apps for a given month
query_7DC = f"""
            select month(review_date) as Month, t2.app_name, count(*) as total_reviews from {table_2} t1
            join {table_1} t2 on t1.app_id = t2.app_id
            group by 1,2;
            """


In [14]:
#Type File
query_1T = f"""
            select 'Apps' as "Type", t1.*, t2.* from {table_1} t1
            join {table_2} t2 on t1.app_id = t2.app_id
            Union All
            select 'Games' as "Type", t1.*, t2.* from {table_3} t1
            join {table_4} t2 on t1.game_id = t2.game_id;
            """

In [16]:
try:
    #connect to MySQL Server
    conn = mysql.connector.connect(
        host = host,
        user = user,
        password = password,
        allow_local_infile = True
    )
    cursor = conn.cursor()

    #Use play_market database
    cursor.execute(f"USE {database_name}")
    print(f"Database Switched!")

    #App Performance & Downloads (Query_1) [Top 10 apps by total downloads (Bar/Column chart : app_name Vs downloads)]
    df1A = pd.read_sql(query_1A, conn)
    df1A.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/App Performance & Downloads/query_1.csv", index = False)
    print("File Exported for query_1A")

    #App Performance & Downloads (Query_2) [Average Score & number of reviews per category (Combo chart: bar for review count + line for avg score)]
    df2A = pd.read_sql(query_2A, conn)
    df2A.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/App Performance & Downloads/query_2.csv", index = False)
    print("File Exported for query_2A")

    #App Performance & Downloads (Query_3) [Monthly trend of app reviews (Line chart - To show distribution over time)]
    df3A = pd.read_sql(query_3A, conn)
    df3A.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/App Performance & Downloads/query_3.csv", index = False)
    print("File Exported for query_3A")

    #App Performance & Downloads (Query_4) [Top 5 most helpful app reviews  (summary table with conditional formatting on Total helpful app reviews)]
    df4A = pd.read_sql(query_4A, conn)
    df4A.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/App Performance & Downloads/query_4.csv", index = False)
    print("File Exported for query_4A")

    #Game Insights (Query_1) [Top 10 games with highest rating and more than 10,000 downloads]
    df1G = pd.read_sql(query_1G, conn)
    df1G.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Game Insights/query_1.csv", index = False)
    print("File Exported for query_1G")

    #Game Insight (Query_2) [Game section with average score and number of games (Clustered bar chart: category vs Average Score and Total Games/Treemap: Size by total_games, color by avg_score)
    df2G = pd.read_sql(query_2G, conn)
    df2G.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Game Insights/query_2.csv", index = False)
    print("File Exported for query_2G")

    #Game Insight (Query_3) [Monthly treand of game reviews (Line chart - To show distribution over time)]
    df3G = pd.read_sql(query_3G, conn)
    df3G.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Game Insights/query_3.csv", index = False)
    print("File Exported for query_3G")

    #time.sleep(2) 

    #User Review Analysis (Query_1) [Apps with the highest average review score (from apps_reviews)]
    df1U = pd.read_sql(query_1U, conn)
    df1U.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/User Review Analysis/query_1.csv", index = False)
    print("File Exported for query_1U")
    
    #User Review Analysis (Query_2) [Total reviews and average helpful count per app/game (Bar Chart: App/Game vs Review Count,Bubble chart: Total Reviews Vs Avg Helpful Count)]
    df2U = pd.read_sql(query_2U, conn)
    df2U.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/User Review Analysis/query_2.csv", index = False)
    print("File Exported for query_2U")
    
    #User Review Analysis (Query_2) [Top apps and games by total review volume (Bar Chart: App/Game name vs total reviews)]
    df3UA = pd.read_sql(query_3UA, conn)
    df3UA.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/User Review Analysis/query_3A.csv", index = False)
    print("File Exported for query_3UA")

    df3UB = pd.read_sql(query_3UB, conn)
    df3UB.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/User Review Analysis/query_3B.csv", index = False)
    print("File Exported for query_3UB")

    #time.sleep(2)1
    
    # Dynamic_Charting (Filter by category: Show App Performance)[Create a drop-down to choose categories. The chart(e.g. bar chart of downloads per app) updates accordingly]
    df1DC = pd.read_sql(query_1DC, conn)
    df1DC.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Dynamic Charting/query_1DC.csv", index = False)
    print("File Exported for query_1DC")

    # Dynamic_Charting (Compare App Review Scores by Section)[Drop down to filter by section. Display a line or column chart of app names vs scores]
    df2DC = pd.read_sql(query_2DC, conn)
    df2DC.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Dynamic Charting/query_2DC.csv", index = False)
    print("File Exported for query_2DC")

    # Dynamic_Charting (Review Trend Over Time by App)[Drop-Down to select app_name, Line chart updates to show monthly review volume]
    df3DC = pd.read_sql(query_3DC, conn)
    df3DC.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Dynamic Charting/query_3DC.csv", index = False)
    print("File Exported for query_3DC")

    # Dynamic_Charting (Select a Game Section and Compare Downloads)[Drop-down with section values. Plot top games by downloads using a bar chart]
    df4DC = pd.read_sql(query_4DC, conn)
    df4DC.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Dynamic Charting/query_4DC.csv", index = False)
    print("File Exported for query_4DC")

    # Dynamic_Charting (Helpful Review Analysis per App/Game)[Drop-down of app names. Gauge chart shows average helpfulness of reviews]
    df5DC_A = pd.read_sql(query_5DC_A, conn)
    df5DC_A.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Dynamic Charting/query_5DC_A.csv", index = False)
    print("File Exported for query_5DC_A")

    # Dynamic_Charting (Download Vs Score Scatter Plot for selected category)[Drop-Down with categories. Scatter plot shows relationship between downloads and score]
    df6DC = pd.read_sql(query_6DC, conn)
    df6DC.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Dynamic Charting/query_6DC.csv", index = False)
    print("File Exported for query_6DC")

    # Dynamic_Charting (Download Vs Score Scatter Plot for selected category)[Drop-Down with categories. Scatter plot shows relationship between downloads and score]
    df7DC = pd.read_sql(query_7DC, conn)
    df7DC.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/New folder/Excel Dashboard Export/Dynamic Charting/query_7DC.csv", index = False)
    print("File Exported for query_7DC")

    # Type_file
    df1T = pd.read_sql(query_1T, conn)
    df7DC.to_csv("C:/Users/anura/OneDrive/Desktop/Play Market/Datasets/Type.csv", index = False)
    print("File Exported for query_1T")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Access denied: Check your username or password.")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist and couldn't be created.")
    else:
        print(err)

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection closed.")

Database Switched!
File Exported for query_1A


C:\Users\anura\AppData\Local\Temp\ipykernel_19936\2861724632.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1A = pd.read_sql(query_1A, conn)
C:\Users\anura\AppData\Local\Temp\ipykernel_19936\2861724632.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2A = pd.read_sql(query_2A, conn)


File Exported for query_2A


C:\Users\anura\AppData\Local\Temp\ipykernel_19936\2861724632.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3A = pd.read_sql(query_3A, conn)


File Exported for query_3A


...

File Exported for query_5DC_A
File Exported for query_6DC


C:\Users\anura\AppData\Local\Temp\ipykernel_19936\2861724632.py:101: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6DC = pd.read_sql(query_6DC, conn)
C:\Users\anura\AppData\Local\Temp\ipykernel_19936\2861724632.py:106: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df7DC = pd.read_sql(query_7DC, conn)


File Exported for query_7DC


C:\Users\anura\AppData\Local\Temp\ipykernel_19936\2861724632.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1T = pd.read_sql(query_1T, conn)


File Exported for query_1T
MySQL connection closed.
